# Overview of pipeline

The purpose of this notebook is to generate entity resolution results for a variety of combinations of Spark NLP for Healthcare models.

The algorithm will accept a dataframe of campaign urls and fund descriptions and will output a long-formatted dataframe with columns specifying each component of the algorithm.

---
### Requirements
- Google Colab Pro with high RAM runtime (Runtime tab -> Change runtime type)





# Import license keys

In [1]:
from google.colab import files
import json

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving keys.json to keys.json


# Install and import dependencies

In [ ]:
import os
import csv
import io
import pandas as pd
import numpy as np
import copy


secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

SparkNLP Version: 2.6.5
SparkNLP-JSL Version: 2.7.2


# Define pipeline elements

In [ ]:
document_assembler = DocumentAssembler() \
  .setInputCol('text')\
  .setOutputCol('document')

sentence_detector = SentenceDetector() \
  .setInputCols(['document'])\
  .setOutputCol('sentence')

tokenizer = Tokenizer()\
  .setInputCols(['sentence']) \
  .setOutputCol('token')

#-------------------------------------------------------------------------------
#embeddings
#-------------------------------------------------------------------------------
 
word_embeddings_clinical = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
#-------------------------------------------------------------------------------


#-------------------------------------------------------------------------------
#NER
#-------------------------------------------------------------------------------

ner_diseases = NerDLModel.pretrained("ner_diseases", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")
#-------------------------------------------------------------------------------


ner_converter_diseases = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['Disease'])


chunk_embeddings = ChunkEmbeddings()\
    .setInputCols(["ner_chunk", "embeddings"])\
    .setOutputCol("chunk_embeddings")

#------------------------
# sentence embeddings
#-------------------------

c2doc = Chunk2Doc().setInputCols("ner_chunk").setOutputCol("ner_chunk_doc") 

sbiobert_embedder = BertSentenceEmbeddings\
  .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
  .setInputCols(["ner_chunk_doc"])\
  .setOutputCol("sbert_embeddings")

sbert_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
  .setInputCols(["ner_chunk", "sbert_embeddings"]) \
  .setOutputCol("entity")\
  .setDistanceFunction("EUCLIDEAN")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_diseases download started this may take some time.
Approximate size to download 13.7 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


# Define functions

In [ ]:
def RunER(ner_result, sent_embeddings, entity_resolution_model):
  '''
  input: spark dataframe of NER results
  output: pandas dataframe with nested results
  '''

  #initiate empty df
  empty_df = spark.createDataFrame([['']]).toDF('fund_description')

  #define pipeline components
  if entity_resolution_model == 'chunkresolve_icd10cm_clinical':
    pipe = Pipeline(stages=[chunk_embeddings, icd10])
  if entity_resolution_model == 'chunkresolve_snomed_findings_clinical':
    pipe = Pipeline(stages=[chunk_embeddings, snomed])
  if entity_resolution_model == 'sbiobertresolve_icd10cm_augmented':
    if sent_embeddings == 'sbiobert_base_cased_mli':
      pipe = Pipeline(stages=[ c2doc, sbiobert_embedder, sbert_resolver])
  
  #run entity resolution
  pipe_model = pipe.fit(empty_df)
  results = pipe_model.transform(ner_result)
  
  return results.toPandas()[['url','fund_description','ner_chunk','entity']]

In [ ]:
def RemoveStopwords(chunks):
  results = []
  for string in chunks:
    stopwords = ['a', 'an', 'the', 'this', 'that', 'these', 'his', 'her', 'their']
    words = string.split()
    resultwords  = [word for word in words if word.lower() not in stopwords]
    r = ' '.join(resultwords)
    results.append(r)
  return results

In [ ]:
def Concatenate_And(chunks):
  return ' and '.join(chunks)

In [ ]:
def PrepareNewDoc(lofd, doc_reconstruction):
  
  #define results container
  r = []

  #concatenate ner terms with 'and'
  if doc_reconstruction == 'concat_and':
    for d in lofd:
      dic = copy.deepcopy(d)
      dic['chunks'] = []
      for chunk in d['ner_chunk']:
        dic['chunks'].append(chunk['result'])
        dic['chunks'] = RemoveStopwords(dic['chunks'])
        dic['new_doc'] = Concatenate_And(dic['chunks'])
      r.append(dic)
    return pd.DataFrame(r)
  
  #select sentences from original document with ner terms
  if doc_reconstruction == 'sent_with_ner':
    r = []
    for d in lofd:
      dic = copy.deepcopy(d)
      ner_chunk = dic['ner_chunk']
      sent_to_include = []
      for chunk in ner_chunk:
        sent_to_include.append(chunk['sentence'])
      dic['sent_to_include'] = sent_to_include
      r.append(dic)

    r2 = []
    for d in r:
      dic = copy.deepcopy(d)
      dic['new_doc'] = ' '.join([dic['sentence'][i]['result'] for i in dic['sent_to_include']])
      r2.append(dic)
    
    return pd.DataFrame(r2)


In [ ]:
def GetChunksAndSentences(lofd):
  results = []
  for d in lofd:
    dic = {}
    dic['url'] = d['url']
    dic['fund_description'] = d['fund_description']
    dic['sentence'] = d['sentence']
    dic['num_sentences'] = len(d['sentence'])
    dic['ner_chunk'] = []
    for chunk in d['ner_chunk']:
      dic['ner_chunk'].append({'result': chunk['result'], 'sentence': int(chunk.metadata['sentence'])})
    results.append(dic)
  return results

In [ ]:
def ReconstructDoc(doc_reconstruction, ner_result, ner_model, first_column):
  '''
  input: document reconstruction parameter, NER results, NER model
  output: NER results based with reconstructed input text
  '''

  #no reconstruction
  if doc_reconstruction == 'none':
    return ner_result
  
  #reconstruct document
  else:
    #clean results
    results_ner_1_df = ner_result.toPandas()
    results_ner_1_lofd = results_ner_1_df.to_dict('records') 
    results_ner_1_lofd_new = GetChunksAndSentences(results_ner_1_lofd)

    #new doc for second round of NER
    new_doc = PrepareNewDoc(results_ner_1_lofd_new, doc_reconstruction)

    #pandas df has some NaN values, convert to empty string
    new_doc = new_doc.fillna('')

    #run second NER
    new_doc = new_doc[['url','new_doc']]
    new_doc.columns = ['url','fund_description']
    ner_result_2 = RunNER(new_doc, ner_model, 'fund_description')

    return ner_result_2

In [ ]:
def RunNER(feed, ner_model, first_column):
  '''
  input:
    1. pandas dataframe of feed data, col_1 = url, col_2 = fund_description
    2. string of ner model
    3. string of first column name
  output: pandas dataframe with nested results
  '''
  #initiate empty df
  empty_df = spark.createDataFrame([['']]).toDF(first_column)
  
  #load feed data into df
  df = spark.createDataFrame(feed)

  #define first entity recognition pipeline with appropriate column name
  document_assembler = DocumentAssembler() \
  .setInputCol(first_column)\
  .setOutputCol('document')
  
  
  if ner_model == 'ner_clinical':

    word_clinical_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence_detector,
      tokenizer,
      word_embeddings_clinical,
      ner_clinical,
      ner_converter_problem])

    ner_clinical_model = word_clinical_pipeline.fit(empty_df)
    result_clinical = ner_clinical_model.transform(df)
    return result_clinical
  
  if ner_model == 'ner_diseases':

    word_diseases_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence_detector,
      tokenizer,
      word_embeddings_clinical,
      ner_diseases,
      ner_converter_diseases])

    ner_diseases_model = word_diseases_pipeline.fit(empty_df)
    result_diseases = ner_diseases_model.transform(df)
    return result_diseases
  
  if ner_model == 'ner_jsl':

    word_jsl_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence_detector,
      tokenizer,
      word_embeddings_clinical,
      ner_jsl,
      ner_converter_diagnosis])

    ner_jsl_model = word_jsl_pipeline.fit(empty_df)
    result_jsl = ner_jsl_model.transform(df)
    return result_jsl

In [ ]:
def RunPipeline(feed, combos):
  '''
  input: list of strings specifying combination of model components
  output: spark nlp pipeline
  '''

  #extract model components
  ner_model = combos[0]
  doc_reconstruction = combos[1]
  sent_embeddings = combos[2]
  er_model = combos[3]

  #run NER
  ner_result = RunNER(feed, ner_model, 'fund_description')

  #reconstruct document
  ner_result_doc = ReconstructDoc(doc_reconstruction, ner_result, ner_model, 'fund_description')

  #run ER
  er = RunER(ner_result_doc, sent_embeddings, er_model)

  #add columns specifying pipeline components
  er['ner_model'] = [ner_model for x in er['url']]
  er['doc_reconstruction'] = [doc_reconstruction for x in er['url']]
  er['sent_embeddings'] = [sent_embeddings for x in er['url']]
  er['er_model'] = [er_model for x in er['url']]
  
  #return results
  return er


In [ ]:
def Main(feed, combo):
  '''
  input: 
    1. pandas dataframe of feed data, col_1 = url, col_2 = fund_description
    2. pandas dataframe of model combinations
  output: long-format pandas dataframes for NER and ER separately
  '''

  #Run model for each combination of model components
  output = RunPipeline(feed, combo)

  #process results
  #split into ner and er
  ner = output.drop(['entity'], axis=1)
  er = output.drop(['ner_chunk'], axis=1)

  #explode dataframes
  ner = ner.explode('ner_chunk')
  er = er.explode('entity')

  #add new data fields
  ner['ner_term'] = [x['result'] if x is not np.nan else np.nan for x in ner['ner_chunk']]
  ner['sentence'] = [x.metadata['sentence'] if x is not np.nan else np.nan for x in ner['ner_chunk']]
  ner['start_char'] = [x['begin'] if x is not np.nan else np.nan for x in ner['ner_chunk']]
  ner['end_char'] = [x['end'] if x is not np.nan else np.nan for x in ner['ner_chunk']]
  ner = ner.drop(['ner_chunk'], axis=1)
  #ner = ner.drop(['fund_description'], axis=1)

  er['target_text'] = [x.metadata['target_text'] if x is not np.nan else np.nan for x in er['entity']]
  er['sentence'] = [x.metadata['sentence'] if x is not np.nan else np.nan for x in er['entity']]
  er['start_char'] = [x['begin'] if x is not np.nan else np.nan for x in er['entity']]
  er['end_char'] = [x['end'] if x is not np.nan else np.nan for x in er['entity']]
  er['resolved_text'] = [x.metadata['resolved_text'] if x is not np.nan else np.nan for x in er['entity']]
  er['entity_code'] = [x['result'] if x is not np.nan else np.nan for x in er['entity']]
  er['confidence'] = [x.metadata['confidence'] if x is not np.nan else np.nan for x in er['entity']]
  er['distance'] = [x.metadata['distance'] if x is not np.nan else np.nan for x in er['entity']]
  #er = er.drop(['entity'], axis=1)
  #er = er.drop(['fund_description'], axis=1)

  #return results
  #return ner, er
  return er

# Import data

Feed Data

In [ ]:
uploaded = files.upload()
round3 = pd.read_csv(io.BytesIO(uploaded['example_101-150.csv']))

Saving example_101-150.csv to example_101-150.csv


In [ ]:
uploaded = files.upload()
round4 = pd.read_csv(io.BytesIO(uploaded['example_151-200.csv']))

Saving example_151-200.csv to example_151-200.csv


In [ ]:
uploaded = files.upload()
round5 = pd.read_csv(io.BytesIO(uploaded['example_201-250.csv']))

Saving example_201-250.csv to example_201-250.csv


In [ ]:
uploaded = files.upload()
round6 = pd.read_csv(io.BytesIO(uploaded['example_251-300.csv']))

Saving example_251-300.csv to example_251-300.csv


In [ ]:
merge = pd.concat([round3, round4, round5, round6])

Model combinations

In [ ]:
uploaded = files.upload()
combinations = pd.read_csv(io.BytesIO(uploaded['model_combinations.csv']))

Saving model_combinations.csv to model_combinations.csv


# Execute functions

In [ ]:
er = Main(merge, combinations.iloc[12].tolist())

NameError: ignored

In [ ]:
combinations

,ner_model,doc_reconstruction,embeddings,entity_resolution_model
0,ner_clinical,none,none,chunkresolve_icd10cm_clinical
1,ner_clinical,concat_and,none,chunkresolve_icd10cm_clinical
2,ner_clinical,sent_with_ner,none,chunkresolve_icd10cm_clinical
3,ner_clinical,none,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented
4,ner_clinical,concat_and,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented
5,ner_clinical,sent_with_ner,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented
6,ner_jsl,none,none,chunkresolve_icd10cm_clinical
7,ner_jsl,concat_and,none,chunkresolve_icd10cm_clinical
8,ner_jsl,sent_with_ner,none,chunkresolve_icd10cm_clinical
9,ner_jsl,none,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented


# Export data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
er.to_csv('/content/drive/My Drive/Crowdfunding/er_rounds_3-6.csv', index=False)

# Model evaluation

In [ ]:
er['score'] = [1 if x is not np.nan else 0 for x in er['target_text']]

In [ ]:
def ModelEval(er, combinations):

  combos = GetCombos(combinations)

  input_docs = []
  ner_terms_count = []
  docs_with_no_ner = []
  ner_terms_per_doc_mean = []
  ner_terms_per_doc_sd = []
  er_confidence_mean = []
  er_distance_mean = []

  for combo in combos:
    df = er.loc[(er['ner_model'] == combo[0]) & (er['doc_reconstruction'] == combo[1]) & (er['sent_embeddings'] == combo[2]) & (er['er_model'] == combo[3]),]
    
    #number of input dociments
    input_docs.append(50)

    #number of NAs for NER terms
    num_na = df['target_text'].isna().sum()

    #total number of NER terms
    ner_terms_count.append(df.shape[0] - num_na)
    
    #NER terms by document
    ner_by_url = df[['url','score']].groupby('url').agg('sum')
    docs_with_no_ner.append(pd.Series(ner_by_url['score'] == 0).sum())
    ner_terms_per_doc_mean.append(ner_by_url['score'].mean())
    ner_terms_per_doc_sd.append(ner_by_url['score'].std())
    
    #ER metrics
    er_confidence_mean.append(df['confidence'].astype(float).mean())
    er_distance_mean.append(df['distance'].astype(float).mean())
  
  r = pd.DataFrame({'input_docs':input_docs,
                    'ner_terms_count':ner_terms_count,
                    'docs_with_no_ner':docs_with_no_ner,
                    'ner_terms_per_doc_mean':ner_terms_per_doc_mean,
                    'ner_terms_per_doc_sd':ner_terms_per_doc_sd,
                    'er_confidence_mean':er_confidence_mean,
                    'er_distance_mean':er_distance_mean})
  return pd.concat([combinations, r], axis=1)
  

In [ ]:
ModelEval(er, combinations)

,ner_model,doc_reconstruction,embeddings,entity_resolution_model,input_docs,ner_terms_count,docs_with_no_ner,ner_terms_per_doc_mean,ner_terms_per_doc_sd,er_confidence_mean,er_distance_mean
0,ner_clinical,none,none,chunkresolve_icd10cm_clinical,50,437,3,8.74,10.866180,0.062511,1.015560
1,ner_clinical,concat_and,none,chunkresolve_icd10cm_clinical,50,413,3,8.26,10.538191,0.065614,0.963053
2,ner_clinical,sent_with_ner,none,chunkresolve_icd10cm_clinical,50,923,3,18.46,31.883212,0.063406,0.972786
3,ner_clinical,none,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented,50,437,3,8.74,10.866180,0.375657,6.381022
4,ner_clinical,concat_and,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented,50,413,3,8.26,10.538191,0.436572,5.756073
5,ner_clinical,sent_with_ner,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented,50,923,3,18.46,31.883212,0.404267,6.015442
6,ner_jsl,none,none,chunkresolve_icd10cm_clinical,50,113,16,2.26,3.983691,0.066368,0.580966
7,ner_jsl,concat_and,none,chunkresolve_icd10cm_clinical,50,103,16,2.06,3.924855,0.066749,0.566989
8,ner_jsl,sent_with_ner,none,chunkresolve_icd10cm_clinical,50,204,16,4.08,10.703499,0.066055,0.596892
9,ner_jsl,none,sbiobert_base_cased_mli,sbiobertresolve_icd10cm_augmented,50,113,16,2.26,3.983691,0.724112,3.072292


Why do some sent_with_ner have like double the numnber of ner terms?